In [158]:
from searchPlus import *

In [159]:
line1 = "## ## ## ## ## ## ## ## ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## .. .. .. 00 .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. .. ##\n"
line7 = "## .. .. .. 01 .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## ## ## ## ## ## ## ## ##\n"
grid = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

In [160]:
class PenguinsPairs(Problem):
    def __init__(self, ice_map=grid):
        self.initial = self.parse_grid(ice_map)
        self.rows = len(self.initial)
        self.cols = len(self.initial[0])
        super().__init__(self.initial)


    def parse_grid(self, ice_map):
        return tuple(tuple(line.split()) for line in ice_map.strip().split("\n"))


    # def actions(self, state):
    #     moves = []
    #     directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'O': (0, -1)}
        
    #     for penguin in [p for row in state for p in row if p.isdigit()]:
    #         for i in range(self.rows):
    #             for j in range(self.cols):
    #                 if state[i][j] == penguin:
    #                     for d, (di, dj) in directions.items():
    #                         ni, nj = i + di, j + dj
    #                         while 0 <= ni < self.rows and 0 <= nj < self.cols and state[ni][nj] == "..":
    #                             ni += di
    #                             nj += dj
    #                         if 0 <= ni < self.rows and 0 <= nj < self.cols and state[ni][nj] not in ('()', penguin):
    #                             moves.append((penguin, d))

    #     return sorted(moves)

    def actions(self, state):
        moves = []
        directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'O': (0, -1)}

        for i in range(self.rows):
            for j in range(self.cols):
                if state[i][j].isdigit() and len(state[i][j]) == 2:  # Знайшли пінгвіна (00-99)
                    penguin = state[i][j]
                    for d, (di, dj) in directions.items():
                        ni, nj = i + di, j + dj

                        # Якщо наступна клітинка одразу ## або (), рух неможливий
                        if not (0 <= ni < self.rows and 0 <= nj < self.cols) or state[ni][nj] in ('##'):
                            continue

                        # Ковзаємо доти, доки не знайдемо ##, (), іншого пінгвіна або межу
                        while 0 <= ni < self.rows and 0 <= nj < self.cols and state[ni][nj] not in ('##'):
                            if state[ni][nj].isdigit() and len(state[ni][nj]) == 2:  # Інший пінгвін
                                break
                            ni += di
                            nj += dj

                        # Остання безпечна позиція перед `##`, `()`, або іншим пінгвіном
                        ni -= di
                        nj -= dj

                        # Якщо кінцева позиція змінилася, додаємо рух
                        if (ni, nj) != (i, j):
                            moves.append((penguin, d))

        return sorted(moves)




    def result(self, state, action):
        penguin, direction = action
        # print(action)
        directions = {'N': (-1, 0), 'S': (1, 0), 'E': (0, 1), 'O': (0, -1)}
        di, dj = directions[direction]
        
        new_state = [list(row) for row in state]
        
        if action in self.actions(state):
            for i in range(self.rows):
                for j in range(self.cols):
                    if new_state[i][j] == penguin:
                        ni, nj = i + di, j + dj
                        while 0 <= ni < self.rows and 0 <= nj < self.cols and new_state[ni][nj] == "..":
                            ni += di
                            nj += dj
                        ni -= di
                        nj -= dj
                        
                        new_state[i][j] = ".."
                        new_state[ni][nj] = penguin
                        return tuple(tuple(row) for row in new_state)
        
        return state


    # def goal_test(self, state):
    #     return any(
    #         # state[i][j] not in ("..", "##", "()") and state[i+1][j] not in ("..", "##", "()") and 
    #         state[i][j].isdigit() and state[i+1][j].isdigit() and state[i][j] != state[i+1][j]
    #         for i in range(self.rows - 1) for j in range(self.cols)
    #     )
    #     # return any(
    #     #     state[i][j] == "00" and state[i+1][j] == "01"
    #     #     for i in range(self.rows - 1) for j in range(self.cols)
    #     # )

    def goal_test(self, state):
        for i in range(self.rows - 1):
            for j in range(self.cols):
                if state[i][j].isdigit() and state[i+1][j].isdigit():
                    first = state[i][j]
                    second = state[i+1][j]
                    
                    if int(first) < int(second):
                        return True
        
        return False


    def display(self, state):
        return "\n".join(" ".join(row) for row in state) + "\n"
    

    def executa(self, state, actions_list, verbose=False):
        """Executa uma sequência de acções a partir do estado devolvendo o triplo formado pelo estado final, 
        pelo custo acumulado e pelo booleano que indica se o objectivo foi ou não atingido. Se o objectivo 
        for atingido antes da sequência ser atingida, devolve-se o estado e o custo corrente.
        Há o modo verboso e o não verboso, por defeito."""
        cost = 0
        for a in actions_list:
            seg = self.result(state, a)
            cost = self.path_cost(cost, state, a, seg)
            state = seg
            obj = self.goal_test(state)
            if verbose:
                print('Ação:', a)
                print(self.display(state), end='')
                print('Custo Total:', cost)
                print('Atingido o objectivo?', obj)
                print()
            if obj:
                break
        return (state, cost, obj)

In [161]:
p = PenguinsPairs(grid)
print(p.actions(p.initial))

[('00', 'E'), ('00', 'N'), ('00', 'O'), ('00', 'S'), ('01', 'E'), ('01', 'N'), ('01', 'O'), ('01', 'S')]


In [162]:
print(p.display(p.initial))

## ## ## ## ## ## ## ## ##
## .. .. .. .. .. .. .. ##
## .. .. .. 00 .. .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. () () () .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. .. 01 .. .. .. ##
## .. .. .. .. .. .. .. ##
## ## ## ## ## ## ## ## ##



In [163]:
line1 = "## () () () () () () () ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## .. .. .. 00 .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. () () () .. .. ##\n"
line6 = "## .. .. .. .. .. .. .. ##\n"
line7 = "## .. .. .. 01 .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## () () () () () () () ##\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

In [164]:
p2 = PenguinsPairs(grid2)
print(p2.actions(p2.initial))

[('00', 'E'), ('00', 'N'), ('00', 'O'), ('00', 'S'), ('01', 'E'), ('01', 'N'), ('01', 'O'), ('01', 'S')]


In [165]:
p = PenguinsPairs(grid)
seq = [('00', 'E'),('01', 'E'),('00', 'S')]
p.goal_test(p.executa(p.initial, seq, verbose=True)[0])

Ação: ('00', 'E')
## ## ## ## ## ## ## ## ##
## .. .. .. .. .. .. .. ##
## .. .. .. .. .. .. 00 ##
## .. .. .. .. .. .. .. ##
## .. .. () () () .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. .. 01 .. .. .. ##
## .. .. .. .. .. .. .. ##
## ## ## ## ## ## ## ## ##
Custo Total: 1
Atingido o objectivo? False

Ação: ('01', 'E')
## ## ## ## ## ## ## ## ##
## .. .. .. .. .. .. .. ##
## .. .. .. .. .. .. 00 ##
## .. .. .. .. .. .. .. ##
## .. .. () () () .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. .. .. .. .. 01 ##
## .. .. .. .. .. .. .. ##
## ## ## ## ## ## ## ## ##
Custo Total: 2
Atingido o objectivo? False

Ação: ('00', 'S')
## ## ## ## ## ## ## ## ##
## .. .. .. .. .. .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. .. .. .. .. .. ##
## .. .. () () () .. .. ##
## .. .. .. .. .. .. 00 ##
## .. .. .. .. .. .. 01 ##
## .. .. .. .. .. .. .. ##
## ## ## ## ## ## ## ## ##
Custo Total: 3
Atingido o objectivo? True



True

In [166]:
# print("00".isdigit())

In [167]:
line1 = "## ## ## ## ## ## ## ## ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## .. .. .. 00 .. .. .. ##\n"
line4 = "## .. .. .. .. .. .. .. ##\n"
line5 = "## .. .. ## ## ## .. .. ##\n"
line6 = "## .. .. .. .. .. .. .. ##\n"
line7 = "## .. .. 01 .. .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## ## ## ## ## ## ## ## ##\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid2)
resultado = breadth_first_graph_search(p)
if resultado:
    print("Solução Larg-prim (grafo) com custo", str(resultado.path_cost) + ":")
    print(resultado.solution())
else:
    print("Sem solução!")

print(p.actions(p.initial))

Solução Larg-prim (grafo) com custo 3:
[('00', 'E'), ('01', 'E'), ('00', 'S')]
[('00', 'E'), ('00', 'N'), ('00', 'O'), ('00', 'S'), ('01', 'E'), ('01', 'N'), ('01', 'O'), ('01', 'S')]


In [168]:
line1 = "## ## ## ## ## ## ## ## ##\n"
line2 = "## .. .. .. .. .. .. .. ##\n"
line3 = "## 00 .. .. .. .. .. .. ##\n"
line4 = "## ## ## ## ## ## ## ## ##\n"
line5 = "## 01 .. .. .. .. .. .. ##\n"
line6 = "## ## ## ## ## ## .. ## ##\n"
line7 = "## .. .. .. .. .. .. .. ##\n"
line8 = "## .. .. .. .. .. .. .. ##\n"
line9 = "## ## ## ## ## ## ## ## ##\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8 + line9

p = PenguinsPairs(grid3)
resultado = breadth_first_graph_search(p)
if resultado:
    print("Solução Larg-prim (grafo) com custo", str(resultado.path_cost) + ":")
    print(resultado.solution())
else:
    print("Sem solução!")

print(p.actions(p.initial))

Sem solução!
[('00', 'E'), ('00', 'N'), ('01', 'E')]
